In [88]:
!pip install rtree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.4 MB/s eta 0:00:00


In [89]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import numpy as np
import dask.dataframe as dd
import dask.distributed
from shapely.geometry import Point, Polygon
import multiprocessing
from tqdm import tqdm
from rtree import index

In [80]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
gdf_hole = pd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/マンホール/yokohama_2018_2022.csv', encoding='cp932')

<ipython-input-81-fef40a7a6534>:1: DtypeWarning: Columns (94,98,101,115) have mixed types. Specify dtype option on import or set low_memory=False.
  gdf_hole = pd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/マンホール/yokohama_2018_2022.csv', encoding='cp932')


In [86]:
gdf_hole[gdf_hole['人孔番号']==2219220074]['geometry_x']

100562    POINT (-17563.941 -61858.540)
Name: geometry_x, dtype: geometry

In [82]:
def parse_point(point_str):
    try:
        #文字列になっているから必要な数値だけ取得（POINT Z）を削除
        point_str = point_str[9:-2]

        #空白で分けx, yに代入
        x_str, y_str = point_str.strip().split()
        #ポイントに変換
        return Point(float(x_str), float(y_str))

        #エラーの場合は欠損地を入力
    except ValueError:
        return np.nan

#各行に対して関数を用いるときはapplyを用いる。Seriesだとエラーが起こることが多い
gdf_hole['geometry_x'] = gdf_hole['geometry_x'].apply(parse_point)

In [83]:
#dataframeをgeodataframeに変換
gdf_hole = GeoDataFrame(gdf_hole, geometry='geometry_x')

In [85]:
#座標系を変更
src_proj = 4326 # 変換前の座標系を指定
dst_proj = 6677  # 変換後の座標系を指定

gdf_hole.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_hole = gdf_hole.to_crs(epsg=dst_proj)  # 変換後座標に変換

##標高結合

In [ ]:
df_elevation = dd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/数値地図/DEM/03215.csv', usecols=['X', 'Y', 'hyoukouchi'])

In [ ]:
# # カラムを絞る
# df_elevation = df_elevation[['X', 'Y', 'hyoukouchi']]

# カラムの名称を変更
df_elevation = df_elevation.rename(columns={'X': 'longitude', 'Y': 'latitude'})


# Pointオブジェクトの作成
geometry = df_elevation.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1, meta=('geometry', 'object')).compute()


# GeoPandasデータフレームを作成
gdf_elevation = gpd.GeoDataFrame(df_elevation.compute(), geometry=geometry)

#gdf_elevationに座標系はついていないので付与
if gdf_elevation.crs is None:
	gdf_elevation.set_crs(epsg=4326, inplace=True)

# 不要なメモリを解放
del df_elevation, geometry

# 標高のデータを結合
gdf_osyu = gpd.sjoin(df_hole, gdf_elevation, how='left', op='contains')

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
#標高の値をグループ演算で平均、最大、最低を求める
gdf_elevation = gdf_osyu.groupby('Meshcode').agg({'hyoukouchi':['mean']}).reset_index()